<a href="https://colab.research.google.com/github/h0806449f/PyTorch/blob/main/DB_06_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **=== 0. 簡介 ===**
1. Get data
2. Create dataset and dataloader
3. Get pre-train model and customize it
4. Train model
5. Evaluate model
6. Make prediction

# **=== 1. Get data ===**

In [78]:
# Import
import requests
import zipfile
import os
from pathlib import Path

# Set path
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# 判斷有無資料夾, 無則創立
if image_path.is_dir():
    print(f"Folder {image_path} already exists.")
else:
    print(f"Foldr {image_path} not exist, creating ...")
    image_path.mkdir(parents = True, exist_ok = True)

# 下載壓縮檔
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading data ...")
    f.write(request.content)

# 解壓縮
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_f:
    print("Unziping data ...")
    zip_f.extractall(image_path)

# 刪除壓縮檔
os.remove(data_path / "pizza_steak_sushi.zip")

Folder data/pizza_steak_sushi already exists.
Unziping data ...


# **=== 2. Dataset & dataloader ===**
1. 可 import 之前的 code, 這邊維持練習
2. **資料預處理的 transform 也需一起遷移學習**

## 2.1 練習

In [79]:
# Directory path
train_dir = image_path / "train"
test_dir = image_path / "test"

In [80]:
# Transforms (此次遷移學習目標為 efficientnet_b0, transform 也需參照)
# 手動練習
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406],
                         std = [0.229, 0.224, 0.225])
])
transform

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [81]:
# Dataset
from torchvision import datasets

train_dataset = datasets.ImageFolder(root = train_dir, transform = transform)
test_dataset = datasets.ImageFolder(root = test_dir, transform = transform)

In [82]:
# Dataloader
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size = 2, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 2, shuffle = False)

## 2.1 遷移: **transform -> dataset -> dataloader**

In [83]:
# Weights
import torchvision

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [84]:
# Transform
from torchvision import transforms

transform = weights.transforms()
transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [85]:
# Dataset
from torchvision import datasets

train_dataset = datasets.ImageFolder(root = train_dir, transform = transform)
test_dataset = datasets.ImageFolder(root = test_dir, transform = transform)

In [86]:
# Dataloader
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size = 2, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 2, shuffle = False)

# **=== 3. Customize pre-trained model ===**
1. Get pre-trained model and check info
2. **Freeze** features layer
3. **Customize** classifier layer

## 3.1 Get pre-trained model

In [87]:
!pip install torchinfo
from torchinfo import summary

In [88]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model_1 = torchvision.models.efficientnet_b0(weights = weights,
                                             progress = False)

summary(model_1,
        input_size = (1, 3, 224, 224),
        col_names=["output_size", "num_params", "trainable"],
        col_width=15)

Layer (type:depth-idx)                                  Output Shape    Param #         Trainable
EfficientNet                                            [1, 1000]       --              True
├─Sequential: 1-1                                       [1, 1280, 7, 7] --              True
│    └─Conv2dNormActivation: 2-1                        [1, 32, 112, 112] --              True
│    │    └─Conv2d: 3-1                                 [1, 32, 112, 112] 864             True
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112] 64              True
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112] --              --
│    └─Sequential: 2-2                                  [1, 16, 112, 112] --              True
│    │    └─MBConv: 3-4                                 [1, 16, 112, 112] 1,448           True
│    └─Sequential: 2-3                                  [1, 24, 56, 56] --              True
│    │    └─MBConv: 3-5                                

## 3.2 Freeze feature layers
model_1 概括架構 : model_1.features /  model_1.avgpool / model_1.classifier

In [89]:
for param in model_1.parameters():
    param.requires_grad = False

In [90]:
summary(model_1,
        input_size = (1, 3, 224, 224),
        col_names=["output_size", "num_params", "trainable"],
        col_width=15)

Layer (type:depth-idx)                                  Output Shape    Param #         Trainable
EfficientNet                                            [1, 1000]       --              False
├─Sequential: 1-1                                       [1, 1280, 7, 7] --              False
│    └─Conv2dNormActivation: 2-1                        [1, 32, 112, 112] --              False
│    │    └─Conv2d: 3-1                                 [1, 32, 112, 112] (864)           False
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112] (64)            False
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112] --              --
│    └─Sequential: 2-2                                  [1, 16, 112, 112] --              False
│    │    └─MBConv: 3-4                                 [1, 16, 112, 112] (1,448)         False
│    └─Sequential: 2-3                                  [1, 24, 56, 56] --              False
│    │    └─MBConv: 3-5                        

## 3.3 Customize classifier layer

In [91]:
# model_1 原輸出層
print(model_1.classifier)

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)


In [92]:
# 更改為符合我們任務的輸出層
import torch

model_1.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p = 0.2),
    torch.nn.Linear(in_features = 1280, out_features = 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(p = 0.2),
    torch.nn.Linear(in_features = 256, out_features = 16),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features = 16, out_features = 3)
)

summary(model_1,
        input_size = (1, 3, 224, 224),
        col_names=["output_size", "num_params", "trainable"],
        col_width=15)

Layer (type:depth-idx)                                  Output Shape    Param #         Trainable
EfficientNet                                            [1, 3]          --              Partial
├─Sequential: 1-1                                       [1, 1280, 7, 7] --              False
│    └─Conv2dNormActivation: 2-1                        [1, 32, 112, 112] --              False
│    │    └─Conv2d: 3-1                                 [1, 32, 112, 112] (864)           False
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112] (64)            False
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112] --              --
│    └─Sequential: 2-2                                  [1, 16, 112, 112] --              False
│    │    └─MBConv: 3-4                                 [1, 16, 112, 112] (1,448)         False
│    └─Sequential: 2-3                                  [1, 24, 56, 56] --              False
│    │    └─MBConv: 3-5                      

# **=== 4. Train model ===**
1. 練習 func: train
2. 練習 func: test
3. 練習 func: train_test_loop
4. Train model

## 4.1 Func: train

In [93]:
# Func: train

## 4.2 Func: test

In [94]:
# Func: test

# 4.3 Func: train_test_loop

In [95]:
# Func: train_test_loop

## 4.4 Train model

# **=== 5. Evaluate model ===**

# **=== 6. Make prediction ===**